# 03 - Federated Learning

## Defines

Define the available types of federated learning.

In [22]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['INDIVIDUAL_ATTACK', 'ATTACK_GROUP', 'STRATIFIED']
METHOD = 'ATTACK_GROUP'
NUM_OF_STRATIFIED_CLIENTS = 10  # only applies to stratified method
NUM_OF_ROUNDS = 10              # Number of FL rounds


Include the defines for the dataframe columns and the attack labels and their mappings

In [23]:
from enum import Enum
from includes import *

##  Imports

In [24]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [25]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


In [26]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.4.0
numpy 1.23.5
torch 2.0.1+cpu
Training on cpu


## Load the Dataset

In [27]:
DATASET_DIRECTORY = '../datasets/CICIoT2023/'

## Training data

Either read the training pickle file if it exists, or process the dataset from scratch.

In [28]:
# Check to see if the file 'training_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    ######################
    # HACK TEMP CODE
    ######################
    # Set training_sets to the last entry of training_sets
    training_sets = training_sets[-5:]
    print(f"HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - {training_sets}")
    ######################
    # HACK END TEMP CODE
    ######################

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


Training sets: 135
Test sets: 34
HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - ['part-00130-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00131-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00132-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00133-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv', 'part-00134-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv']
Reading training data...


100%|██████████| 5/5 [00:06<00:00,  1.31s/it]


Writing training data to pickle file...
Training data size: (1425287, 47)


In [29]:
train_df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,182.00,17.00,64.00,22.362751,22.362751,0.0,0.0,0.0,0.0,...,0.000000,182.00,8.300743e+07,9.5,19.078784,0.000000,0.000000,0.00,141.55,13
1,2.437778,129.60,6.00,64.00,0.978382,0.978382,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336252e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,7
2,0.000000,54.00,6.00,64.00,0.000000,0.000000,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334496e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,1
3,0.453670,39173.00,17.00,64.00,4967.422026,4967.422026,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.310643e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4
4,0.000000,54.00,6.00,64.00,166.930829,166.930829,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331469e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425282,0.000000,54.00,6.00,64.00,19.582485,19.582485,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331443e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
1425283,0.037146,78.22,36.21,63.18,24.542045,24.542045,0.0,0.0,0.0,0.0,...,110.233513,453.78,8.358187e+07,9.5,30.338676,154.660856,23401.960226,0.53,141.55,18
1425284,3.293075,1025996.92,17.00,64.00,572.160392,572.160392,0.0,0.0,0.0,0.0,...,0.000000,554.00,8.378910e+07,9.5,33.286634,0.000000,0.000000,0.00,141.55,19
1425285,0.047343,35223.00,17.00,64.00,15083.107398,15083.107398,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.309852e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4


---
## Test Data
Concat the test data into a single dataframe

In [30]:
# Check to see if the file 'test_data.pkl' exists in the directory. If it does, load it. If not, print an error.
testing_data_pickle_file = 'testing_data.pkl'

if os.path.isfile(testing_data_pickle_file):
    print(f"File {testing_data_pickle_file} exists, loading data...")
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")

else:
    print(f"File {testing_data_pickle_file} does not exist, constructing data...")

    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    ############################################
    ############################################
    # HACK - Make things quicker for now
    ############################################
    ############################################

    test_sets = df_sets[int(len(df_sets)*.95):]
    
    # Set training_sets to the last entry of training_sets
    test_sets = test_sets[-5:]
    
    ############################################
    ############################################
    # END HACK 
    ############################################
    ############################################

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    # Concatenate all testing sets into one dataframe
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print(f"Writing test data to pickle file {testing_data_pickle_file}...")
    test_df.to_pickle(testing_data_pickle_file)

print("Testing data size: {}".format(test_df.shape))

File testing_data.pkl does not exist, constructing data...
Test sets: 5
Reading test data...


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:07<00:00,  1.46s/it]


Writing test data to pickle file testing_data.pkl...
Testing data size: (1803497, 47)


---
# Scale the test and train data

### Scale the training data input features

In [31]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

### Scale the testing data input features

In [32]:
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---
# Define the classification problem - (2 classes, 8 classes or 34 classes)
Change the following cell to select the classification type

If the METHOD == STRATIFIED, then we can use any classifier
If the METHOD == ATTACK_GROUP then we must use Group Classifier.

In [33]:
binary_classifier = False
group_classifier = True
individual_classifier = False

In [34]:

class_size_map = {2: "Binary", 8: "Group", 34: "Individual"}

if group_classifier:
    print("Group 8 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_7_classes values
    test_df['label'] = test_df['label'].map(dict_8_classes)
    train_df['label'] = train_df['label'].map(dict_8_classes)
    class_size = "8"
        
elif binary_classifier:
    print("Binary 2 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_2_classes values
    test_df['label'] = test_df['label'].map(dict_2_classes)
    train_df['label'] = train_df['label'].map(dict_2_classes)
    class_size = "2"

else:
    print ("Individual 34 Class classifier... - No adjustments to labels in test and train dataframes")
    class_size = "34"

Group 8 Class Classifier... - Adjusting labels in test and train dataframes


---
# Split the Training Data into partitions for the Federated Learning clients depending on the test required

In [35]:
from sklearn.model_selection import StratifiedKFold

# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

if METHOD == 'STRATIFIED':
    print(f"{Colours.YELLOW.value}STRATIFIED METHOD{Colours.NORMAL.value}")
    # We are going to split the training data into 'NUM_OF_STRATIFIED_CLIENTS' smaller groups using StratifiedKFold
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
        fl_X_train.append(train_df[X_columns].iloc[test_index])
        fl_y_train.append(train_df[y_column].iloc[test_index])

elif METHOD == 'ATTACK_GROUP':
    print(f"{Colours.YELLOW.value}ATTACK_GROUP METHOD{Colours.NORMAL.value}")
    # With this method we split the data so that each client data sees all attacks except one.
    # EG:
    # client 0 will see attacks 1-7
    # client 1 will see attacks 0,2-7
    # client 2 will see attacks 0-1,3-7
    
    # There are 8 attack groups, so we will create 8 clients
    skf = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        # Create a new dataframe for the client data
        client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i]], ignore_index=True)
        fl_X_train.append(client_df[X_columns])
        fl_y_train.append(client_df[y_column])
        
    pass  



ATTACK_GROUP METHOD


C:\Users\jd7-white\AppData\Local\Temp\ipykernel_25840\3250502811.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i]], ignore_index=True)
C:\Users\jd7-white\AppData\Local\Temp\ipykernel_25840\3250502811.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i]], ignore_index=True)
C:\Users\jd7-white\AppData\Local\Temp\ipykernel_25840\3250502811.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i]], ignore_index=True)
C:\Users\jd7-white\AppData\Local\Temp\ipykernel_25840\3250502811.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i]], ignore_index=True)
C:\Users\jd7-white\A

In [36]:
for i in range(len(fl_X_train)):
    # Show the unique values in the y column
    print(f"fl_X_train[{i}].shape: {fl_X_train[i].shape}")  
    print(f"fl_y_train[{i}].value_counts():\n{fl_y_train[i].value_counts()}")
    print(f"fl_y_train[{i}].unique(): {fl_y_train[i].unique()}")

fl_X_train[0].shape: (173976, 46)
fl_y_train[0].value_counts():
1    129710
7     30902
2     10011
4      1829
3      1374
5       100
6        50
Name: label, dtype: int64
fl_y_train[0].unique(): [1 7 2 3 4 5 6]
fl_X_train[1].shape: (48451, 46)
fl_y_train[1].value_counts():
7    30902
2    10011
0     4185
4     1829
3     1374
5       99
6       51
Name: label, dtype: int64
fl_y_train[1].unique(): [7 2 3 0 4 5 6]
fl_X_train[2].shape: (168150, 46)
fl_y_train[2].value_counts():
1    129710
7     30902
0      4185
4      1830
3      1373
5        99
6        51
Name: label, dtype: int64
fl_y_train[2].unique(): [1 7 4 0 3 5 6]
fl_X_train[3].shape: (176788, 46)
fl_y_train[3].value_counts():
1    129710
7     30902
2     10011
0      4185
4      1830
5        99
6        51
Name: label, dtype: int64
fl_y_train[3].unique(): [1 7 4 2 0 6 5]
fl_X_train[4].shape: (176332, 46)
fl_y_train[4].value_counts():
1    129709
7     30902
2     10012
0      4186
3      1373
5        99
6        51
Name

### Convert the training dataset

In [37]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

---
### Data check

In [38]:
NUM_OF_CLIENTS = len(fl_X_train)
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()
print("Original train_df size: {}".format(train_df.shape))

print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 8
NUM_ROUNDS: 10

Original train_df size: (1425287, 47)
Checking training data split groups
0 : X Shape (173976, 46) Y Shape (173976,)
1 : X Shape (48451, 46) Y Shape (48451,)
2 : X Shape (168150, 46) Y Shape (168150,)
3 : X Shape (176788, 46) Y Shape (176788,)
4 : X Shape (176332, 46) Y Shape (176332,)
5 : X Shape (178062, 46) Y Shape (178062,)
6 : X Shape (178110, 46) Y Shape (178110,)
7 : X Shape (147258, 46) Y Shape (147258,)

Checking testing data
X_test size: (1803497, 46)
y_test size: (1803497,)

Deploy Simulation


----
# Federated Learning
## Import the libraries and print the versions

In [39]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

label = train_df[y_column]

Define the Client and Server code

In [40]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

import datetime

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=10, batch_size=64)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=64)
        print(f"{Colours.YELLOW.value}Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}{Colours.NORMAL.value}")

        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(len(label.unique()), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print (f"Server Evaluating... Evaluation Count:{eval_count}")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print(f"{Colours.YELLOW.value}Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}{Colours.NORMAL.value}")
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(len(label.unique()), activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

scikit-learn 1.2.2.
flwr 1.4.0
numpy 1.23.5
tf 2.12.0


In [41]:
%%time
print (f"\n{Colours.YELLOW.value} Deploy simulation... {class_size_map[len(label.unique())]} ({class_size}) Classifier\n{Colours.NORMAL.value}")

start_time = datetime.datetime.now()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

end_time = datetime.datetime.now()
print("Total time taken: ", end_time - start_time)

INFO flwr 2023-07-08 19:09:27,686 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)



 Deploy simulation... Group (8) Classifier



2023-07-08 19:09:35,901	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-08 19:09:40,044 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 11629085492.0, 'object_store_memory': 5814542745.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'CPU': 12.0}
INFO flwr 2023-07-08 19:09:40,045 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-08 19:09:40,046 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-08 19:09:46,625 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-08 19:09:46,626 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=10964) Client ID: 6
Server Evaluating... Evaluation Count:0
56360/56360 [==============================] - 44s 775us/step


INFO flwr 2023-07-08 19:11:41,611 | server.py:91 | initial parameters (loss, other metrics): 2.5491700172424316, {'accuracy': 0.016968701034784317}
INFO flwr 2023-07-08 19:11:41,612 | server.py:101 | FL starting
DEBUG flwr 2023-07-08 19:11:41,613 | server.py:218 | fit_round 1: strategy sampled 8 clients (out of 8)


...

(launch_and_fit pid=10964) 1926/2783 [===================>..........] - ETA: 2s - loss: 0.0243 - accuracy: 0.9908 [repeated 21x across cluster]
(launch_and_fit pid=10964)  451/2783 [===>..........................] - ETA: 6s - loss: 0.0313 - accuracy: 0.9887 [repeated 5x across cluster]
(launch_and_fit pid=10964)  544/2783 [====>.........................] - ETA: 6s - loss: 0.0294 - accuracy: 0.9894 [repeated 6x across cluster]
(launch_and_fit pid=10964)  737/2783 [======>.......................] - ETA: 6s - loss: 0.0286 - accuracy: 0.9897 [repeated 5x across cluster]
(launch_and_fit pid=10964)  821/2783 [=======>......................] - ETA: 5s - loss: 0.0281 - accuracy: 0.9899 [repeated 4x across cluster]
(launch_and_fit pid=10964)  921/2783 [========>.....................] - ETA: 5s - loss: 0.0272 - accuracy: 0.9902 [repeated 5x across cluster]
(launch_and_fit pid=10964) 1010/2783 [=========>....................] - ETA: 4s - loss: 0.0268 - accuracy: 0.9903 [repeated 4x across cluster

INFO flwr 2023-07-08 19:47:03,982 | server.py:119 | fit progress: (10, 0.04279197007417679, {'accuracy': 0.9890113472938538}, 2122.3652891000092)
DEBUG flwr 2023-07-08 19:47:03,983 | server.py:168 | evaluate_round 10: strategy sampled 4 clients (out of 8)


Prediction:  [[2.72272971e-18 9.99996185e-01 1.10235013e-19 ... 7.26672791e-24
  8.26665080e-28 3.80133633e-06]
 [0.00000000e+00 1.00000000e+00 3.24190028e-36 ... 0.00000000e+00
  0.00000000e+00 1.03893329e-19]
 [2.99607983e-23 9.99992013e-01 4.70327549e-23 ... 2.29040203e-30
  7.99692995e-33 7.99270947e-06]
 ...
 [6.14728238e-28 3.85889798e-05 1.63482757e-25 ... 1.94265302e-37
  1.70105695e-37 9.99961376e-01]
 [1.17088155e-23 9.99992728e-01 6.63271157e-23 ... 8.80036385e-31
  1.10459055e-33 7.32875196e-06]
 [0.00000000e+00 1.00000000e+00 1.29068490e-37 ... 0.00000000e+00
  0.00000000e+00 1.53559416e-14]] (1803497, 8)
Server evaluation complete - Accuracy: 0.9890, Loss: 0.0428
(launch_and_evaluate pid=10964) Client ID: 1
(launch_and_fit pid=10964) 1373/2783 [=============>................] - ETA: 3s - loss: 0.0253 - accuracy: 0.9906 [repeated 10x across cluster]
(launch_and_fit pid=10964) 1655/2783 [================>.............] - ETA: 2s - loss: 0.0246 - accuracy: 0.9907 [repeated 1

DEBUG flwr 2023-07-08 19:47:08,748 | server.py:182 | evaluate_round 10 received 4 results and 0 failures
INFO flwr 2023-07-08 19:47:08,749 | server.py:147 | FL finished in 2127.131541499999
INFO flwr 2023-07-08 19:47:08,749 | app.py:218 | app_fit: losses_distributed [(1, 1.693544892401673), (2, 0.9473254070098172), (3, 0.15652556194840703), (4, 0.0970633144709287), (5, 0.04548980258485049), (6, 0.04359935590688885), (7, 0.07736421894397595), (8, 0.14826393233833104), (9, 0.04593962646936413), (10, 0.049489715180227996)]
INFO flwr 2023-07-08 19:47:08,750 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-07-08 19:47:08,751 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-07-08 19:47:08,752 | app.py:221 | app_fit: losses_centralized [(0, 2.5491700172424316), (1, 1.3596932888031006), (2, 1.12362539768219), (3, 0.1483074426651001), (4, 0.10564694553613663), (5, 0.09560389071702957), (6, 0.03886088356375694), (7, 0.09380090236663818), (8, 0.1937430202960968), (9,

Total time taken:  0:37:41.069870
CPU times: total: 28min 57s
Wall time: 37min 41s
